# library


In [80]:
import numpy as np
import random

DAUCAU = ",;-.?!:…"
DAUCAU_POS = ",,......"
DAUCAU_POS = ",;-.?!:…"
DAUCAU_POS = "." * len(DAUCAU)


# input and out processing
def words_to_word( words ):
	return "_".join(words)
def preprocess_sen(sen):
	sen = sen.lower()
	sen = sen.replace("%", "")
	sen = sen.replace("\n", "")

	
	for c in DAUCAU:
		sen = sen.replace(c, " "+c+" ")

	return sen
def sen_to_words( sen ):

	def fullOfSpace( str ):
		for c in str:
			if c != ' ':
				return False 
		
		return True 

	sen = preprocess_sen( sen )

	sen_words = [ w for w in sen.split(" ") if not fullOfSpace(w) ]

	
	return sen_words
def words_and_pos_to_output_sen( sen_words, sen_pos ):
	output = ""
	n = len( sen_words)
	comma = ','
	dot = '.'

	# hide pos of ',' and '.'
	for i in range(n ):
		if i ==0 or sen_words[i] in (comma, dot):
			output += sen_words[i]
		else:
			output += ' ' + sen_words[i]
		
		if not sen_words[i] in (comma,dot):
			output += "/" + sen_pos[i].upper()



	return output
def file_to_tagged_data( path ):
	file = open(path, "r", encoding="utf8")
	text = file.readlines()

	tagged_data = []

	for line in text:
		tagged_line = []

		line_words = sen_to_words( line )

		for w in line_words:
			if w in DAUCAU:
				DAUCAU_index = DAUCAU.index(w)
				tagged_line.append((w,DAUCAU_POS[DAUCAU_index]))
			else:
				s = w.split("/")
				tagged_line.append((s[0], s[1]))

		tagged_data.append( tagged_line )

	return tagged_data

#tach tu bang thuat toan Longest matching
def Longest_matching( sen, words_list):
	words = sen_to_words(sen)
	tokens = []
	n = len( words )
	s = 0
	while s < n :
		e = n
		word = words_to_word( words[s:e] )

		while ( not word in words_list ) and e>s+1:
			e -= 1
			word = words_to_word( words[s:e] )

		tokens.append( word )
		if e <= s :
			s += 1
		else:
			s = e

	return tokens

#gan nhan tu loai tuyen tinh
def find_pos_for_words( words, hidden_markov_matrix, hidden_to_observed_matrix ):
	n = len(words)
	return_pos = []
	return_pro = 1
	for i in range( n ):
		w = words[i]
		if w in DAUCAU:
			DAUCAU_index = DAUCAU.index(w)
			return_pos.append(DAUCAU_POS[DAUCAU_index])
		else:
			accept_tags_list = [ tag for tag in tags_list if tag not in DAUCAU ]
			pro = 0
			accept_pos = ""
			for pos in accept_tags_list:
				temp_pro = 1
				#calculate prepos-pos
				x_index = tags_list.index( pos )
				y_index = 0
				if i == 0:
					y_index = len(tags_list)
				else:
					y_index = tags_list.index( return_pos[i-1] )

				temp_pro *= hidden_markov_matrix[y_index,x_index]
				#calculate pos-word
				y_index = tags_list.index( pos )
				x_index = 0
				if w in words_list:
					x_index = words_list.index(w)
				else:
					x_index = len(words_list)

				temp_pro *= hidden_to_observed_matrix[y_index, x_index]

				if temp_pro > pro:
					pro = temp_pro
					accept_pos = pos
			
			return_pos.append( accept_pos )
			return_pro *= pro

	return return_pos, return_pro

#gan nhan tu loai quy hoach dong (de quy) #CANCELED
def Probality_words_pos( sen_words, sen_pos, hidden_markov_matrix, hidden_to_observed_matrix ):
	if len(sen_words) != len(sen_pos):
		return 0

	pro = 1
	for i in range( len( sen_pos)):
		x_index = tags_list.index( sen_pos[i] )
		y_index = 0
		if i == 0:
			y_index = len( tags_list)
		else:
			y_index = tags_list.index( sen_pos[i-1] )

		pro *= hidden_markov_matrix[y_index,x_index]

	for word, pos in zip( sen_words, sen_pos ):
		y_index = tags_list.index(pos)
		x_index = 0
		if word in words_list:
			x_index = words_list.index( word )
		else:
			x_index = len(words_list)

		pro *= hidden_to_observed_matrix[y_index, x_index]

	return pro
def recur_find_pos_for_words( words, hidden_markov_matrix, hidden_to_observed_matrix ):
	n = len( words )

	def recur_find_pos( pos_list,words, n, hidden_markov_matrix, hidden_to_observed_matrix):
		# stop
		if len(pos_list) == n:
			return pos_list, Probality_words_pos( words, pos_list, hidden_markov_matrix, hidden_to_observed_matrix )
		
		pro = 0
		return_pos_list = []
		word_index = len(pos_list)

		accept_tags_list = []
		if words[ word_index ] not in DAUCAU:
			accept_tags_list = [ tag for tag in tags_list if tag not in DAUCAU ]
		else:
			accept_tags_list = [words[word_index]]

		for pos in accept_tags_list:
			new_pos_list, new_pro = recur_find_pos( pos_list[:] + [pos], words, n, hidden_markov_matrix, hidden_to_observed_matrix)
			if new_pro > pro:
				return_pos_list = new_pos_list
				pro = new_pro
		
		return return_pos_list, pro
	
	return recur_find_pos([], words, n, hidden_markov_matrix, hidden_to_observed_matrix)

	
def get_words_count_from_tagged_data(tagged_data):
	words = {} 
	for line in tagged_data:
		for w, _ in line :
			if not w in words.keys():
				words[w] = 1
			else:
				words[w] += 1
	
	return words
def get_tags_count_from_tagged_data(tagged_data):
	words = {} 
	for line in tagged_data:
		for _, w in line :
			if not w in words.keys():
				words[w] = 1
			else:
				words[w] += 1
	
	return words
def get_only_tags_from_tagged_data ( tagged_data ):
	tags = []
	for line in tagged_data :
		tags_line = []
		for _, tag in line:
			tags_line.append(tag)
		
		tags.append( tags_line )
	
	return tags
def get_only_words_from_tagged_data ( tagged_data ):
	tags = []
	for line in tagged_data :
		tags_line = []
		for tag,_ in line:
			tags_line.append(tag)
		
		tags.append( tags_line )
	
	return tags
def get_tags_list_from_tagged_data( tagged_data ):
	tags = []

	for line in tagged_data:
		for _, tag in line :
			if not tag in tags:
				tags.append(tag)
	
	return tags
def get_words_list_from_tagged_data( tagged_data ):
	words = []
	for line in tagged_data:
		for w, _ in line :
			if not w in words:
				words.append(w)
	
	return words

def Accuracy_on_test_data( test_data ):
	test_pos = get_only_tags_from_tagged_data(test_data)
	test_words = get_only_words_from_tagged_data( test_data )

	total_n_pos = 0
	for sen_pos in test_pos:
		total_n_pos += len(sen_pos)

	test_sens = []
	for sen_words in test_words:
		sen = " ".join(sen_words)
		sen = sen.replace("_"," ")
		test_sens.append( sen )


	correct = 0
	for i in range( len( test_sens )):
		print("--------------------------------")
		test_sen_words = Longest_matching( test_sens[i], words_list )
		result_pos, pro = find_pos_for_words( test_sen_words, hidden_markov_matrix, hidden_to_observed_matrix )
		print(words_and_pos_to_output_sen( test_sen_words, result_pos))
		for j in range( len(test_pos[i]) ):
			try:
				if result_pos[j] == test_pos[i][j] and result_pos[j] not in DAUCAU:
					correct += 1
			except:
				pass

	return correct/ total_n_pos

# code


## data

In [81]:
tagged_data = file_to_tagged_data( "train_data.txt")
test_data = file_to_tagged_data( "test_data.txt")

In [82]:
train_words_count = get_words_count_from_tagged_data(tagged_data)
print(train_words_count)
print( len(train_words_count))

{'gia_đình': 7, 'là': 20, 'tổ_ấm': 1, 'hạnh_phúc': 1, '.': 48, 'hôm_nay': 2, 'trời': 1, 'đẹp': 3, 'quá': 1, 'tôi': 11, 'thua_thiệt': 1, 'với': 9, 'bạn_bè': 1, 'ba': 2, 'người': 9, 'đàn_ông': 1, 'trụ_cột': 1, 'trong': 12, 'bố_mẹ': 1, 'tuyệt_vời': 1, 'nhất': 1, 'đối_với': 3, 'chúng_ta': 1, 'hoa_hồng': 1, 'một': 12, 'loài_hoa': 1, 'yêu': 1, 'quê_hương': 1, ',': 43, 'đất_nước': 2, 'tiếng_việt': 1, 'cực_kỳ': 1, 'phong_phú': 1, 'có_thể': 2, 'nói': 4, 'phức_tạp': 1, 'cấu_tạo': 1, 'và': 11, 'cả': 2, 'ngữ_nghĩa': 1, 'ai': 1, 'cũng': 3, 'có': 4, 'tổ_tiên': 4, 'cội_nguồn': 1, 'để': 7, 'nhớ': 1, 'máy_tính': 1, 'rất': 3, 'quan_trọng': 2, 'đặc_biệt': 1, 'sinh_viên': 1, 'xe_máy': 1, 'phương_tiên': 1, 'lưu_thông': 1, 'còn': 3, 'điện_thoại': 1, 'phương_tiện': 1, 'liên_lạc': 1, 'chàng_trai': 1, 'bơ_vơ': 1, 'từ': 2, 'xa': 1, 'tim': 1, 'hụt_hẫn': 1, 'như': 1, 'mất': 3, 'thứ': 1, 'gì': 1, 'chăm_chỉ': 1, 'đọc': 1, 'sách': 1, 'hoàn_thiện': 1, 'bản_thân': 2, 'học_tập': 1, 'học_hỏi': 1, 'không': 6, 'ngừng': 1,

In [83]:
train_tags_count = get_tags_count_from_tagged_data(tagged_data)
print(train_tags_count)
print( len(train_tags_count))

{'n': 210, 'v': 105, 'a': 52, '.': 95, 'w': 18, 'p': 47, 'q': 76, 'm': 19, 't': 14, 'd': 15, 'l': 23, 'c': 4, 'i': 4}
13


## hidden markov


In [84]:
#  hidden markov matrix
#
#  ---------------------------
#  	   |tag0|tag1|...|tagN|
#  --------+-------------------
#  tag0    |    |    |   |    |
#  --------+----+----+---+----+
#  ...     |    |    |   |    |
#  --------+----+----+---+----+
#  tagN    |    |    |   |    |
#  --------+----+----+---+----+
#  <s>     |    |    |   |    |
#  ----------------------------


#  hidden to observed matrix
#
#  --------------------------------------+
#  	   |word0|word1|...|wordM|unknown|
#  --------+-----+-----+---+-----+-------+
#  tag0    |     |     |   |     |       |
#  --------+-----+-----+---+-----+-------+
#  ...     |     |     |   |     |       |
#  --------+-----+-----+---+-----+-------+
#  tagN    |     |     |   |     |       |
#  --------+-----+-----+---+-----+-------+

In [85]:
# get words list (dictionary), tags list, tags chains
words_list = get_words_list_from_tagged_data( tagged_data )+ ["unknown"]
tags = get_only_tags_from_tagged_data( tagged_data )
tags_list = get_tags_list_from_tagged_data(tagged_data)


# create new matrix reference to hidden markov chain
n = len(tags_list)
hidden_markov_matrix = np.zeros((n+1,n))
for line in tags:
	i = 0
	n = len( line )
	while i < n:
		if i == 0:
			x_index = tags_list.index(line[i])
			y_index = len(tags_list)
		else:
			x_index = tags_list.index(line[i])
			y_index = tags_list.index(line[i-1])

		hidden_markov_matrix[y_index,x_index] += 1
		i+=1 

# create matrix reference to hidden-to-observed connect
n_tags = len(tags_list)
n_words = len(words_list)

hidden_to_observed_matrix = np.zeros((n_tags,n_words+1))
for line in tagged_data:
	for w, tag in line :
		y_index = tags_list.index(tag)
		x_index = words_list.index(w)

		hidden_to_observed_matrix[y_index,x_index] += 1



## smooth and calculate probality

In [86]:
#smooth
hidden_markov_matrix = hidden_markov_matrix + 0.1
hidden_to_observed_matrix = hidden_to_observed_matrix + 0.1

#calculate probality
for i in range( hidden_markov_matrix.shape[0] ):
	total = np.sum( hidden_markov_matrix[i] )
	hidden_markov_matrix[i] = hidden_markov_matrix[i] / total

for i in range( hidden_to_observed_matrix.shape[0] ):
	total = np.sum( hidden_to_observed_matrix[i] )
	hidden_to_observed_matrix[i] = hidden_to_observed_matrix[i] / total


# find pos

In [87]:
# example
sen = "Tôi yêu bóng đá và đất nước"
sen_words = Longest_matching( sen , words_list)
sen_pos, probality = find_pos_for_words( sen_words, hidden_markov_matrix, hidden_to_observed_matrix )

print( sen_words )
print( sen_pos )
print(words_and_pos_to_output_sen( sen_words, sen_pos ))

['tôi', 'yêu', 'bóng_đá', 'và', 'đất_nước']
['n', 'v', 'n', 'q', 'n']
tôi/N yêu/V bóng_đá/N và/Q đất_nước/N


# accuracy

In [88]:
Accuracy_on_test_data( test_data )

--------------------------------
hôm_nay/W gia_đình/N hạnh_phúc/A.
--------------------------------
anh/N chỉ/P thích/V em/N thôi/A.
--------------------------------
tôi/N đang/P chơi/V bóng_đá/N.
--------------------------------
tôi/N yêu/V hoa_hồng/N.
--------------------------------
tôi/N thường_xuyên/W ăn/V dưa_hấu/N.
--------------------------------
học_tập/V để/Q bản_thân/N không/P thua_thiệt/V với/Q bạn_bè/N.
--------------------------------
tôi/N ghét/A chơi/V bóng/P.
--------------------------------
ai/D cũng/P tắm/V.
--------------------------------
tết_nguyên_đán/N là/V ngày/N đoàn_tụ/V gia_đình/N.
--------------------------------
đây/D là/V khoảng/L thời_gian/N thiêng_liêng/A của/Q người_việt/N nam/A.
--------------------------------
mọi/L người/N trong/Q gia_đình/N chúc_tết/V ông_bà/N.
--------------------------------
ngày/N để/Q tưởng_nhớ/V những/L người_thân/N đã/P mất/V.
--------------------------------
thầy_thuốc/N không_bao_giờ/P lựa_chọn/V món_ăn/N ngon/A.
----------

0.7768595041322314